# Advanced Image Processing
- Image segmentation, Distance transforms, Registration
- Real examples: Object segmentation, Image alignment

In [1]:
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
print('Advanced image processing module loaded')

Advanced image processing module loaded


## Thresholding Segmentation
**Simple method**: Separate foreground from background

**Types**:
- Global: Single threshold for entire image
- Adaptive: Local thresholds
- Otsu: Automatic threshold selection

In [2]:
# Grayscale image
img = np.random.rand(200, 200) * 100
y, x = np.ogrid[:200, :200]
img[(y-100)**2 + (x-100)**2 < 40**2] += 100

# Simple threshold
threshold = 100
binary = img > threshold

print(f'Thresholding:')
print(f'  Threshold value: {threshold}')
print(f'  Foreground pixels: {binary.sum()}')
print(f'  Background pixels: {(~binary).sum()}')

Thresholding:
  Threshold value: 100
  Foreground pixels: 5013
  Background pixels: 34987


## Distance Transform
For each foreground pixel, compute distance to nearest background

**Uses**:
- Skeletonization (medial axis)
- Watershed seed points
- Shape analysis

In [3]:
# Binary object
y, x = np.ogrid[:200, :200]
obj = (y-100)**2 + (x-100)**2 < 50**2

# Distance transform
dist = ndimage.distance_transform_edt(obj)

print(f'Distance transform:')
print(f'  Max distance (radius): {dist.max():.1f} pixels')
print(f'  Center point distance: {dist[100, 100]:.1f}')
print(f'
Skeletonization: Points where distance is local maximum')

SyntaxError: unterminated f-string literal (detected at line 11) (3358457289.py, line 11)

## Watershed Segmentation
Separate touching objects
**Method**: Treat image as topographic map, find watershed lines

In [ ]:
# Two touching circles
img_touching = np.zeros((200, 300), dtype=bool)
img_touching[(y-100)**2 + (x-100)**2 < 45**2] = True
img_touching[(y-100)**2 + (x-180)**2 < 45**2] = True

print('Watershed segmentation:')
print('  1. Distance transform')
print('  2. Find local maxima (seeds)')
print('  3. Watershed from seeds')
print('  Result: Separate touching objects')

# Distance transform
dist_touching = ndimage.distance_transform_edt(img_touching)

# Local maxima as markers
local_max = dist_touching > 30
markers, n_markers = ndimage.label(local_max)
print(f'  Detected {n_markers} objects')

## Image Registration
Align two images

**Methods**:
- Rigid: Translation + rotation
- Affine: + scaling + shearing
- Non-rigid: Local deformations

**Uses**: Medical imaging, satellite, panoramas

In [ ]:
# Create reference and shifted image
ref_img = np.zeros((200, 200))
ref_img[80:120, 80:120] = 1

# Shifted version
shifted = ndimage.shift(ref_img, shift=[10, 15])

print('Image registration:')
print('  Reference image created')
print('  Shifted version: offset=(10, 15)')

# Cross-correlation for registration
from scipy import signal
corr = signal.correlate2d(ref_img, shifted, mode='same')
shift_y, shift_x = np.unravel_index(corr.argmax(), corr.shape)
detected_shift = (shift_y - 100, shift_x - 100)

print(f'  Detected shift: {detected_shift}')
print(f'  Error: {np.abs(detected_shift[0]-10)} pixels (y), {np.abs(detected_shift[1]-15)} pixels (x)')

## Real Example: Satellite Image Change Detection
Detect changes between two time points
Applications: Urban growth, deforestation, disaster

In [ ]:
# Simulate satellite images (before and after)
np.random.seed(42)
img_before = np.random.rand(300, 300) * 0.3  # Dark (vegetation)

# Add buildings in 'before'
img_before[50:100, 50:100] = 0.8
img_before[150:180, 200:250] = 0.8

# 'After': New buildings added
img_after = img_before.copy()
img_after[200:240, 80:140] = 0.8  # New building
img_after[100:130, 200:230] = 0.8  # Another new

# Change detection
difference = np.abs(img_after - img_before)
changes = difference > 0.2

# Remove small noise
changes_clean = ndimage.binary_opening(changes, iterations=2)

# Label changed regions
labeled_changes, n_changes = ndimage.label(changes_clean)

print('Satellite change detection:')
print(f'  Time period: Before → After')
print(f'  Changed regions: {n_changes}')
print(f'  Total changed area: {changes_clean.sum()} pixels')

# Measure each change
for i in range(1, n_changes + 1):
    change_mask = labeled_changes == i
    area = change_mask.sum()
    center = ndimage.center_of_mass(change_mask)
    print(f'  Change {i}: Area={area} px, Location={center}')

## Image Restoration
Recover degraded images

**Problems**:
- Blur (defocus, motion)
- Noise
- Missing data

**Methods**:
- Deconvolution (Wiener filter)
- Inpainting (fill missing)
- Super-resolution

In [ ]:
# Blurred image
original = np.zeros((100, 100))
original[40:60, 40:60] = 1

# Motion blur kernel
kernel = np.zeros((15, 15))
kernel[7, :] = 1 / 15  # Horizontal blur

# Convolve to create blur
from scipy.signal import convolve2d
blurred = convolve2d(original, kernel, mode='same')

print('Image restoration:')
print('  Original sharp image')
print('  Applied motion blur')
print('  Goal: Deconvolution to recover')
print('
Wiener deconvolution can partially recover')

## Summary

### Advanced Techniques:
```python
# Thresholding
binary = img > threshold

# Distance transform
dist = ndimage.distance_transform_edt(binary)

# Watershed (conceptual)
# 1. Distance transform
# 2. Find local maxima → markers
# 3. Apply watershed

# Registration (cross-correlation)
from scipy.signal import correlate2d
corr = correlate2d(img1, img2, mode='same')
shift = np.unravel_index(corr.argmax(), corr.shape)
```

### Applications:

**Medical Imaging**:
- Organ segmentation
- Tumor detection
- Multi-modal registration (CT+MRI)

**Satellite/Remote Sensing**:
- Land classification
- Change detection
- Disaster monitoring

**Manufacturing**:
- Defect detection
- Quality control
- Part inspection

**Security**:
- Face detection
- Object tracking
- Anomaly detection

### Best Practices:
✓ **Preprocess first**: Denoise, enhance contrast  
✓ **Choose right method**: Threshold vs watershed  
✓ **Post-process**: Morphology to clean results  
✓ **Validate**: Check segmentation accuracy  
✓ **Tune parameters**: Test on representative data  